In [1]:
import glob

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path
import re
import glob

import pylab as p
from tqdm import tqdm
import gzip
import random
import numpy as np
from collections import defaultdict
from scipy.sparse import csr_matrix, diags, lil_matrix
from scipy.sparse.linalg import eigs

np.random.RandomState(42)

RandomState(MT19937) at 0x132252840

In [2]:
# Ugly duplication of code
def load_data(filepath, headers=None, sep="\t", bad_lines="warn"):
    """
    Loads clickstream data and performs basic cleaning (lowercasing and dropping NaNs).
    :param bad_lines: Defines what to do when encountering a line with too many fields
    :type bad_lines: str (default: "warn")
    :param sep: Delimiter in file.
    :type sep: str
    :param headers: Column headers for file.
    :type headers: list (default: clickstream headers)
    :param filepath: Path to file.
    :type filepath: str
    :return: Cleaned data loaded from given file.
    :rtype: pandas dataframe
    """
    if headers is None:
        headers = ["prev", "curr", "type", "n"]
    filepath = str(filepath)
    if filepath.endswith(".gz"):
        with gzip.open(filepath, "rt", encoding="utf-8") as file_obj:
            clickstream = pd.read_csv(
                file_obj, sep=sep, names=headers, on_bad_lines=bad_lines
            )
    else:
        clickstream = pd.read_csv(
            filepath, sep=sep, names=headers, on_bad_lines=bad_lines
        )
    cols = clickstream.columns
    for col in tqdm(cols, desc="Processing text"):
        try:
            if clickstream[col].dtype == "O":
                clickstream.loc[:, col] = clickstream[col].str.lower()
        except AttributeError as e:
            print(f"Check for non-string datatypes: {e}")
    clickstream.dropna(inplace=True)
    return clickstream


def get_paths(directory, pattern):
    """
    Creates list of filepaths matching the given pattern in the given directory.
    :param directory: Directory containing files of interest.
    :type directory: str
    :param pattern: Pattern of filenames of interest.
    :type pattern: str
    :return: All filenames matching the given pattern in the given directory.
    :rtype: list of strings
    """
    directory_path = Path(directory)
    return directory_path.glob(pattern)


def build_graph(df, from_node="prev", to_node="curr", weight="n"):
    """
    Creates directed graph from given dataframe, with edge weights equal to clicks.
    :param df: Pages with clickstream data.
    :type df: Dataframe
    :param from_node: Name of column containing click origin pages
    :type from_node: str
    :param to_node: Name of column where containing click destination pages
    :type to_node: str
    :param weight: Number of clicks
    :type weight: int
    :return: Clickstream graph
    :rtype: NetworkX object
    """
    g = nx.DiGraph()
    from_nodes = list(df[from_node])
    to_nodes = list(df[to_node])
    weights = list(df[weight])

    for fnode, tnode, wt in zip(from_nodes, to_nodes, weights):
        if fnode == tnode:
            print(f" WARN: self-edge {fnode, tnode}")
        g.add_edge(fnode, tnode, weight=wt)
    return g


def get_node_map(g):
    node_mapping = {node: i for i, node in enumerate(g.nodes)}
    return node_mapping


def get_state_dis(transition_matrix):
    """
    Assumes equal probability at initial state. Not sure I need this function?
    :param transition_matrix:
    :type transition_matrix:
    :return:
    :rtype:
    """
    state_dis = np.full(
        (1, transition_matrix.shape[0]), (1 / transition_matrix.shape[0])
    )
    return state_dis


def get_stationary_dis(transition_matrix):
    """
    https://phys.libretexts.org/Bookshelves/Mathematical_Physics_and_Pedagogy/Computational_Physics_(Chong)/08%3A_Sparse_Matrices/8.03%3A_Using_Sparse_Matrices
    :param transition_matrix: column stochastic
    :type transition_matrix:
    :return:
    :rtype:
    """
    eig_vals, eig_vecs = eigs(transition_matrix)
    stationary_idx = np.where(np.isclose(eig_vals, 1))[0][0]
    stationary_vec = eig_vecs[:, stationary_idx].real

    # Normalize vector
    stationary_vec = np.abs(stationary_vec)
    stationary_vec /= np.sum(stationary_vec)
    return stationary_vec


# def get_transition_matrix(g):
#     num_nodes = g.number_of_nodes()
#     node_mapping = get_node_map(g)
#     row_idx = []
#     col_idx = []
#     data = []

#     for node in g.nodes:
#         out_edges = list(g.out_edges(node))
#         if len(out_edges) == 0:
#             out_edges = [(node, other_node) for other_node in g.nodes]
#             out_weights = [1.0 / num_nodes for _ in range(num_nodes)]
#         else:
#             out_weights = [g.edges[edge]["weight"] for edge in out_edges]
#         total_weights = sum(out_weights)
#         if total_weights > 0:
#             normalized_weights = [w / total_weights for w in out_weights]
#             row_idx.extend([node_mapping[node]] * len(out_edges))
#             col_idx.extend([node_mapping[j] for i, j in out_edges])
#             data.extend(normalized_weights)
#     trans_matrix = csr_matrix(
#         (data, (row_idx, col_idx)), shape=(num_nodes, num_nodes), dtype=np.float64
#     )

#     # column_sums = trans_matrix.sum(axis=0)
#     # sinks = np.asarray(column_sums == 0).flatten()
#     # if np.any(sinks):
#     #     uniform_vec = np.full(num_nodes, 1 / num_nodes)
#     #     sink_locs = np.where(sinks)[0]
#     #     for loc in sink_locs:
#     #         trans_matrix[:,loc] = uniform_vec

#     # Make column stochastic
#     col_stoch_trans_matrix = trans_matrix.transpose()
#     return col_stoch_trans_matrix


# def get_transition_matrix(g):
#     num_nodes = g.number_of_nodes()
#     node_mapping = {node: i for i, node in enumerate(g.nodes)}
#     row_idx = []
#     col_idx = []
#     data = []
# 
#     for node in g.nodes:
#         out_edges = list(g.out_edges(node))
#         out_weights = [g.edges[edge]["weight"] for edge in out_edges]
#         total_weights = sum(out_weights)
#         if total_weights > 0:
#             normalized_weights = [w / total_weights for w in out_weights]
#             row_idx.extend([node_mapping[node]] * len(out_edges))
#             col_idx.extend([node_mapping[j] for i, j in out_edges])
#             data.extend(normalized_weights)
#     trans_matrix = csr_matrix((data, (row_idx, col_idx)), shape=(num_nodes, num_nodes), dtype=np.float64)
#     column_sums = trans_matrix.sum(axis=0)
#     sinks = np.asarray(column_sums == 0).flatten()
#     if np.any(sinks):
#         uniform_vec = np.full(num_nodes, 1 / num_nodes)
#         sink_locs = np.where(sinks)[0]
#         for loc in sink_locs:
#             trans_matrix[:,loc] = uniform_vec
# 
#     # Make column stochastic
#     col_stoch_trans_matrix = trans_matrix.transpose()
#     return col_stoch_trans_matrix

# def get_transition_matrix(g):
#     num_nodes = g.number_of_nodes()
#     node_mapping = get_node_map(g)
#     row_idx = []
#     col_idx = []
#     data = []
#     
#     for node in g.nodes:
#         out_edges = list(g.out_edges(node))
#         out_weights = [g.edges[edge]["weight"] for edge in out_edges]
#         total_weights = sum(out_weights)
#         if total_weights > 0:
#             normalized_weights = [w / total_weights for w in out_weights]
#             row_idx.extend([node_mapping[node]] * len(out_edges))
#             col_idx.extend([node_mapping[j] for i, j in out_edges])
#             data.extend(normalized_weights)
#         # Set teleport probabilities for sinks
#         else:
#             row_idx.extend([node_mapping[node]] for _ in range(num_nodes))
#             col_idx.extend(range(num_nodes))
#             data.extend([1/num_nodes]*num_nodes)
#     trans_matrix = csr_matrix((data, (row_idx, col_idx)), shape=(num_nodes, num_nodes), dtype=np.float64)
#     
#     # Check for sinks and replace with 1/num_nodes:
#     # column_sums = trans_matrix.sum(axis=0)
#     # sinks = np.asarray(column_sums == 0).flatten()
#     # uniform_prob = 1 / num_nodes
#     # trans_matrix = trans_matrix.tolil()  
#     # trans_matrix[:, sinks] = uniform_prob
#     # trans_matrix = trans_matrix.tocsr()
# 
#     # Make column stochastic
#     col_stoch_trans_matrix = trans_matrix.transpose()
#     return col_stoch_trans_matrix

# def get_transition_matrix(g):
#     num_nodes = g.number_of_nodes()
#     node_mapping = {node: i for i, node in enumerate(g.nodes)}
#     row_idx = []
#     col_idx = []
#     data = []
#     for node in g.nodes:
#         out_edges = list(g.out_edges(node))
#         out_weights = [g.edges[edge]["weight"] for edge in out_edges]
#         total_weights = sum(out_weights)
#         if total_weights > 0:
#             normalized_weights = [w / total_weights for w in out_weights]
#             row_idx.extend([node_mapping[node]] * len(out_edges))
#             col_idx.extend([node_mapping[j] for i, j in out_edges])
#             data.extend(normalized_weights)
#     trans_matrix = csr_matrix((data, (row_idx, col_idx)), shape=(num_nodes, num_nodes), dtype=np.float64)
#     # Handle sink nodes
#     column_sums = trans_matrix.sum(axis=0)
#     sinks = np.asarray(column_sums == 0).flatten()
#     if np.any(sinks):
#         uniform_prob = 1 / num_nodes
#         sink_adjustment = csr_matrix((uniform_prob, (np.arange(num_nodes), np.where(sinks)[0])),
#                                      shape=(num_nodes, num_nodes))
#         trans_matrix += sink_adjustment
#     # Ensure the matrix is column stochastic
#     col_sums = np.array(trans_matrix.sum(axis=0)).flatten()
#     col_stoch_trans_matrix = trans_matrix / col_sums
#     return col_stoch_trans_matrix.transpose()


def get_transition_matrix(g):
    # Add node to distribute sinks uniformly
    g.add_node("sink_bridge")
    node_mapping = get_node_map(g)
    num_nodes = g.number_of_nodes()
    
    row_idx = []
    col_idx = []
    data = []
    
    for node in g.nodes:
        out_edges = list(g.out_edges(node))
        out_weights = [g.edges[edge]["weight"] for edge in out_edges]
        total_weights = sum(out_weights)
        
        if total_weights > 0:
            normalized_weights = [w / total_weights for w in out_weights]
            row_idx.extend([node_mapping[node]] * len(out_edges))
            col_idx.extend([node_mapping[j] for i, j in out_edges])
            data.extend(normalized_weights)
        else:
            g.add_edge(node, 'sink_bridge', weight=1)
            row_idx.append(node_mapping[node])
            col_idx.append(node_mapping['sink_bridge'])
            data.append(1)
    for node in g.nodes:
        if node != 'sink_bridge':
            g.add_edge('sink_bridge', node, weight=1/num_nodes)
            row_idx.append(node_mapping['sink_bridge'])
            col_idx.append(node_mapping[node])
            data.append(1/num_nodes)
    trans_matrix = csr_matrix((data, (row_idx, col_idx)), shape=(num_nodes, num_nodes), dtype=np.float64)
    return trans_matrix.transpose()


def test_transition_matrix(transition_matrix):
    column_sums = transition_matrix.sum(axis=0)
    non_zero_cols = column_sums > 0
    sinks = column_sums == 0
    print(f"Sinks: {np.count_nonzero(sinks)}")
    print(f"Non-zero cols: {np.count_nonzero(non_zero_cols)}")
    print("Non-zero cols are stochastic:")
    return np.allclose(
        column_sums[non_zero_cols], np.ones(column_sums[non_zero_cols].shape), atol=1e-6
    )


def random_walk(
    g, trans_matrix, stationary_matrix, start, max_length=1e5, converge_threshold=1e-5
):
    # Initialize dict of visited pages
    visited = defaultdict(int)

    # Set starting node
    node_list = list(g.nodes())
    start_page = node_list.index(start)
    page = np.zeros(len(node_list))
    page[start_page] = 1

    # Initialize progress bar
    pbar = tqdm(total=max_length)
    pbar.set_description("Random Walker Progress")

    # Run random walk until convergence or max iterations reached
    iteration = 0
    while True:
        transition_probs = trans_matrix.dot(page)
        # Test normalizing transition_probs
        transition_probs /= transition_probs.sum()
        visit = np.random.choice(len(page), p=transition_probs)
        visited[visit] += 1
        page = np.zeros_like(page)
        page[visit] = 1
        iteration += 1
        pbar.update(1)

        if iteration % 100 == 0 or iteration >= max_length:
            curr_state = page / page.sum()
            error = np.abs(curr_state - stationary_matrix).sum()

            if error < converge_threshold or iteration >= max_length:
                break
    pbar.close()
    return visited, visit

## Test Random Walker

In [3]:
subset = pd.read_pickle("../all_nodes.pkl")
wiki_201901 = load_data("../data/2017-2023/clickstream-enwiki-2019-01.tsv.gz")

Processing text: 100%|██████████| 4/4 [00:09<00:00,  2.30s/it]


In [4]:
filter_201901 = wiki_201901[
    (wiki_201901["prev"].isin(page.lower() for page in subset))
    & (wiki_201901["curr"].isin(page.lower() for page in subset))
]

In [5]:
graph_201901 = build_graph(filter_201901)
len(graph_201901)

 WARN: self-edge ('the_oa', 'the_oa')
 WARN: self-edge ('grand_theft_auto', 'grand_theft_auto')
 WARN: self-edge ('canteen', 'canteen')
 WARN: self-edge ('vic', 'vic')
 WARN: self-edge ('fear_of_the_dark', 'fear_of_the_dark')
 WARN: self-edge ('lala', 'lala')
 WARN: self-edge ('bet', 'bet')
 WARN: self-edge ('raptor', 'raptor')
 WARN: self-edge ('ys', 'ys')
 WARN: self-edge ('sugar_baby', 'sugar_baby')
 WARN: self-edge ('awal', 'awal')
 WARN: self-edge ('genx', 'genx')
 WARN: self-edge ('python', 'python')
 WARN: self-edge ('spice', 'spice')
 WARN: self-edge ("death's_head", "death's_head")
 WARN: self-edge ('nxt', 'nxt')
 WARN: self-edge ('total_depravity', 'total_depravity')
 WARN: self-edge ('jamaica_station', 'jamaica_station')
 WARN: self-edge ('bec', 'bec')
 WARN: self-edge ('deadbeef', 'deadbeef')
 WARN: self-edge ('y_combinator', 'y_combinator')
 WARN: self-edge ('tops', 'tops')
 WARN: self-edge ('ariel', 'ariel')
 WARN: self-edge ('clip', 'clip')
 WARN: self-edge ('philadelphi

243640

In [6]:
random.seed(42)
random_start = random.choice(list(graph_201901.nodes()))
random_start

'the_broken_star'

In [7]:
trans_matrix = get_transition_matrix(graph_201901)

In [8]:
test_transition_matrix(trans_matrix)

Sinks: 0
Non-zero cols: 243641
Non-zero cols are stochastic:


False

In [9]:
trans_matrix.sum(axis=0).sum()

243641.9999958956

In [10]:
trans_matrix.shape

(243641, 243641)

In [11]:
stationary = get_stationary_dis(trans_matrix)
stationary

array([6.10806979e-20, 3.07155632e-21, 7.42694142e-21, ...,
       2.21213797e-21, 1.22073429e-21, 2.46113681e-16])

In [ ]:
random_walk(graph_201901, trans_matrix, stationary, random_start)

Random Walker Progress:   7%|▋         | 71055/1000000.0 [11:06<2:25:56, 106.09it/s]